# Microsoft Agent Framework을 활용한 Human-in-the-Loop 워크플로우

## 🎯 학습 목표

이 노트북에서는 Microsoft Agent Framework의 `RequestInfoExecutor`를 사용하여 **human-in-the-loop** 워크플로우를 구현하는 방법을 배웁니다. 이 강력한 패턴은 AI 워크플로우를 일시 중지하여 인간의 입력을 수집할 수 있게 하며, 에이전트를 상호작용 가능하게 만들고 중요한 결정에 대해 인간이 통제할 수 있도록 합니다.

## 🔄 Human-in-the-Loop란 무엇인가?

**Human-in-the-loop (HITL)**은 AI 에이전트가 실행을 일시 중지하고 인간의 입력을 요청한 후 계속 진행하는 디자인 패턴입니다. 이는 다음과 같은 상황에서 필수적입니다:

- ✅ **중요한 결정** - 중요한 행동을 취하기 전에 인간의 승인을 받음
- ✅ **모호한 상황** - AI가 불확실할 때 인간이 명확히 할 수 있도록 함
- ✅ **사용자 선호도** - 여러 옵션 중에서 사용자가 선택하도록 요청
- ✅ **규정 준수 및 안전** - 규제된 작업에 대해 인간의 감독을 보장
- ✅ **상호작용 경험** - 사용자 입력에 반응하는 대화형 에이전트 구축

## 🏗️ Microsoft Agent Framework에서의 작동 방식

프레임워크는 HITL을 위한 세 가지 주요 구성 요소를 제공합니다:

1. **`RequestInfoExecutor`** - 워크플로우를 일시 중지하고 `RequestInfoEvent`를 방출하는 특별한 실행자
2. **`RequestInfoMessage`** - 인간에게 보내는 유형화된 요청 페이로드의 기본 클래스
3. **`RequestResponse`** - `request_id`를 사용하여 인간의 응답과 원래 요청을 연결

**워크플로우 패턴:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 예제: 사용자 확인이 포함된 호텔 예약

조건부 워크플로우를 기반으로 하여 대체 목적지를 제안하기 **전에** 사용자 확인을 추가합니다:

1. 사용자가 목적지를 요청 (예: "파리")
2. `availability_agent`가 객실 가능 여부를 확인
3. **객실이 없을 경우** → `confirmation_agent`가 "대체 목적지를 보시겠습니까?"라고 질문
4. 워크플로우가 **`RequestInfoExecutor`를 사용하여 일시 중지**
5. **인간이 응답** - 콘솔 입력을 통해 "예" 또는 "아니오"를 선택
6. `decision_manager`가 응답에 따라 경로를 결정:
   - **예** → 대체 목적지 표시
   - **아니오** → 예약 요청 취소
7. 최종 결과 표시

이 예제는 에이전트의 제안에 대해 사용자가 통제권을 가질 수 있도록 하는 방법을 보여줍니다!

---

시작해봅시다! 🚀


## 1단계: 필요한 라이브러리 가져오기

표준 에이전트 프레임워크 구성 요소와 **사람이 개입하는 특정 클래스**를 가져옵니다:
- `RequestInfoExecutor` - 사람의 입력을 위해 워크플로를 일시 중지하는 실행기
- `RequestInfoEvent` - 사람의 입력이 요청될 때 발생하는 이벤트
- `RequestInfoMessage` - 유형화된 요청 페이로드의 기본 클래스
- `RequestResponse` - 요청과 사람의 응답을 연결
- `WorkflowOutputEvent` - 워크플로 출력 감지를 위한 이벤트


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## 2단계: 구조화된 출력에 대한 Pydantic 모델 정의

이 모델들은 에이전트가 반환할 **스키마**를 정의합니다. 조건부 워크플로우의 모든 모델을 유지하고 다음을 추가합니다:

**Human-in-the-Loop에 새로 추가된 항목:**
- `HumanFeedbackRequest` - 인간에게 전송되는 요청 페이로드를 정의하는 `RequestInfoMessage`의 서브클래스
  - `prompt`(질문 내용)과 `destination`(사용할 수 없는 도시와 관련된 컨텍스트)를 포함합니다


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## 3단계: 호텔 예약 도구 만들기

조건부 워크플로우에서 사용한 동일한 도구 - 목적지에서 객실이 이용 가능한지 확인합니다.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## 4단계: 라우팅을 위한 조건 함수 정의

휴먼 인 더 루프 워크플로우를 위해 **네 가지 조건 함수**가 필요합니다:

**조건부 워크플로우에서:**
1. `has_availability_condition` - 호텔이 **있을 때** 라우팅
2. `no_availability_condition` - 호텔이 **없을 때** 라우팅

**휴먼 인 더 루프를 위해 새로 추가된 조건:**
3. `user_wants_alternatives_condition` - 사용자가 대안을 **원한다고** 답할 때 라우팅
4. `user_declines_alternatives_condition` - 사용자가 대안을 **원하지 않는다고** 답할 때 라우팅


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## 5단계: Decision Manager Executor 생성하기

이 단계는 **사람이 참여하는 패턴의 핵심**입니다! `DecisionManager`는 사용자 정의 `Executor`로서 다음을 수행합니다:

1. `RequestResponse` 객체를 통해 **사람의 피드백을 수신**합니다.
2. 사용자의 결정(예/아니오)을 **처리**합니다.
3. 적절한 에이전트에게 메시지를 보내 **워크플로를 라우팅**합니다.

주요 기능:
- `@handler` 데코레이터를 사용하여 메서드를 워크플로 단계로 노출합니다.
- 원래 요청과 사용자의 답변을 모두 포함하는 `RequestResponse[HumanFeedbackRequest, str]`를 수신합니다.
- 우리의 조건 함수들을 트리거하는 간단한 "예" 또는 "아니오" 메시지를 생성합니다.


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## 6단계: 사용자 지정 디스플레이 실행기 생성

조건부 워크플로에서 사용된 동일한 디스플레이 실행기 - 워크플로 출력으로 최종 결과를 제공합니다.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## 7단계: 환경 변수 로드

LLM 클라이언트(GitHub Models, Azure OpenAI, 또는 OpenAI)를 구성합니다.


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## 8단계: AI 에이전트와 실행자 생성

**여섯 가지 워크플로우 구성 요소**를 만듭니다:

**에이전트 (AgentExecutor로 감싸짐):**
1. **availability_agent** - 도구를 사용하여 호텔 가용성을 확인합니다.
2. **confirmation_agent** - 🆕 사용자 확인 요청을 준비합니다.
3. **alternative_agent** - 대체 도시를 제안합니다 (사용자가 '예'라고 말할 때).
4. **booking_agent** - 예약을 권장합니다 (객실이 있을 때).
5. **cancellation_agent** - 🆕 취소 메시지를 처리합니다 (사용자가 '아니오'라고 말할 때).

**특별 실행자:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`는 사용자 입력을 위해 워크플로우를 일시 중지합니다.
7. **decision_manager** - 🆕 사용자 응답에 따라 경로를 지정하는 맞춤 실행자 (위에서 이미 정의됨).


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## 9단계: 인간 개입이 포함된 워크플로우 구축

이제 **조건부 라우팅**을 포함하여 인간 개입 경로를 포함한 워크플로우 그래프를 구성합니다:

**워크플로우 구조:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**주요 경로:**
- `availability_agent → confirmation_agent` (객실이 없을 때)
- `confirmation_agent → prepare_human_request` (유형 변환)
- `prepare_human_request → request_info_executor` (인간 개입을 위해 대기)
- `request_info_executor → decision_manager` (항상 - RequestResponse 제공)
- `decision_manager → alternative_agent` (사용자가 "예"라고 말할 때)
- `decision_manager → cancellation_agent` (사용자가 "아니오"라고 말할 때)
- `availability_agent → booking_agent` (객실이 있을 때)
- 모든 경로는 `display_result`에서 종료됩니다


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## 10단계: 테스트 케이스 1 실행 - 가용성이 없는 도시 (인간 확인이 필요한 파리)

이 테스트는 **인간이 개입하는 전체 주기를** 보여줍니다:

1. 파리에서 호텔 요청
2. availability_agent 확인 → 객실 없음
3. confirmation_agent가 인간을 대상으로 하는 질문 생성
4. request_info_executor가 **워크플로를 일시 중지**하고 `RequestInfoEvent`를 발생시킴
5. **애플리케이션이 이벤트를 감지하고 콘솔에서 사용자에게 알림**
6. 사용자가 "yes" 또는 "no"를 입력
7. 애플리케이션이 `send_responses_streaming()`을 통해 응답 전송
8. decision_manager가 응답에 따라 라우팅
9. 최종 결과 표시

**핵심 패턴:**
- 첫 번째 반복에서는 `workflow.run_stream()` 사용
- 이후 반복에서는 `workflow.send_responses_streaming(pending_responses)` 사용
- 인간 입력이 필요한 경우를 감지하려면 `RequestInfoEvent`를 청취
- 최종 결과를 캡처하려면 `WorkflowOutputEvent`를 청취


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## 11단계: 테스트 케이스 2 실행 - 이용 가능한 도시 (스톡홀름 - 사용자 입력 불필요)

이 테스트는 **직접 경로**를 보여줍니다. 객실이 이용 가능한 경우:

1. 스톡홀름에서 호텔 요청
2. availability_agent 확인 → 객실 이용 가능 ✅
3. booking_agent가 예약을 제안
4. display_result가 확인 메시지 표시
5. **사용자 입력 불필요!**

객실이 이용 가능한 경우, 워크플로는 사용자 개입 경로를 완전히 우회합니다.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## 주요 요점 및 Human-in-the-Loop 모범 사례

### ✅ 배운 내용:

#### 1. **RequestInfoExecutor 패턴**
Microsoft Agent Framework의 human-in-the-loop 패턴은 세 가지 주요 구성 요소를 사용합니다:
- `RequestInfoExecutor` - 워크플로를 일시 중지하고 이벤트를 발생시킴
- `RequestInfoMessage` - 유형화된 요청 페이로드의 기본 클래스 (이를 서브클래싱하세요!)
- `RequestResponse` - 인간의 응답과 원래 요청을 연관시킴

**핵심 이해:**
- `RequestInfoExecutor`는 자체적으로 입력을 수집하지 않습니다 - 단지 워크플로를 일시 중지할 뿐입니다.
- 애플리케이션 코드는 `RequestInfoEvent`를 수신하고 입력을 수집해야 합니다.
- `request_id`를 사용자의 답변에 매핑하는 딕셔너리를 사용하여 `send_responses_streaming()`을 호출해야 합니다.

#### 2. **스트리밍 실행 패턴**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **이벤트 기반 아키텍처**
특정 이벤트를 수신하여 워크플로를 제어합니다:
- `RequestInfoEvent` - 인간 입력이 필요함 (워크플로 일시 중지)
- `WorkflowOutputEvent` - 최종 결과가 사용 가능함 (워크플로 완료)
- `WorkflowStatusEvent` - 상태 변경 (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS 등)

#### 4. **@handler를 사용한 사용자 정의 Executor**
`DecisionManager`는 다음을 보여줍니다:
- `@handler` 데코레이터를 사용하여 메서드를 워크플로 단계로 노출
- 유형화된 메시지 수신 (예: `RequestResponse[HumanFeedbackRequest, str]`)
- 다른 executor로 메시지를 보내 워크플로 라우팅
- `WorkflowContext`를 통해 컨텍스트에 접근

#### 5. **인간 결정에 따른 조건부 라우팅**
인간의 응답을 평가하는 조건 함수를 생성할 수 있습니다:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 실제 응용 사례:

1. **승인 워크플로**
   - 경비 보고서를 처리하기 전에 관리자 승인을 받음
   - 자동 이메일을 보내기 전에 인간 검토 필요
   - 고가치 거래를 실행하기 전에 확인

2. **콘텐츠 검토**
   - 의심스러운 콘텐츠를 인간 검토 대상으로 플래그 지정
   - 경계 사례에 대해 최종 결정을 내리도록 모더레이터에게 요청
   - AI 신뢰도가 낮을 때 인간에게 에스컬레이션

3. **고객 서비스**
   - AI가 일상적인 질문을 자동으로 처리
   - 복잡한 문제를 인간 상담원에게 에스컬레이션
   - 고객에게 인간과 대화하고 싶은지 묻기

4. **데이터 처리**
   - 모호한 데이터 항목을 해결하도록 인간에게 요청
   - 불명확한 문서에 대한 AI 해석 확인
   - 여러 유효한 해석 중에서 사용자가 선택하도록 허용

5. **안전 필수 시스템**
   - 되돌릴 수 없는 작업 전에 인간 확인 필요
   - 민감한 데이터에 접근하기 전에 승인 받기
   - 규제 산업(의료, 금융)에서 결정 확인

6. **인터랙티브 에이전트**
   - 후속 질문을 하는 대화형 봇 구축
   - 사용자를 복잡한 프로세스를 안내하는 마법사 생성
   - 인간과 단계별로 협력하는 에이전트 설계

### 🔄 비교: Human-in-the-Loop 유무

| 기능 | 조건부 워크플로 | Human-in-the-Loop 워크플로 |
|------|----------------|---------------------------|
| **실행** | 단일 `workflow.run()` | `run_stream()` + `send_responses_streaming()` 루프 |
| **사용자 입력** | 없음 (완전 자동화) | `input()` 또는 UI를 통한 대화형 프롬프트 |
| **구성 요소** | Agents + Executors | + RequestInfoExecutor + DecisionManager |
| **이벤트** | AgentExecutorResponse만 | RequestInfoEvent, WorkflowOutputEvent 등 |
| **일시 중지** | 일시 중지 없음 | RequestInfoExecutor에서 워크플로 일시 중지 |
| **인간 제어** | 인간 제어 없음 | 인간이 주요 결정 |
| **사용 사례** | 자동화 | 협업 및 감독 |

### 🚀 고급 패턴:

#### 다중 인간 결정 지점
하나의 워크플로에서 여러 `RequestInfoExecutor` 노드를 가질 수 있습니다:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### 타임아웃 처리
인간 응답에 대한 타임아웃 구현:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### 풍부한 UI 통합
`input()` 대신 웹 UI, Slack, Teams 등과 통합:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### 조건부 Human-in-the-Loop
특정 상황에서만 인간 입력 요청:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ 모범 사례:

1. **항상 RequestInfoMessage를 서브클래싱**
   - 타입 안전성과 유효성 검사 제공
   - UI 렌더링을 위한 풍부한 컨텍스트 활성화
   - 각 요청 유형의 의도를 명확히 함

2. **설명적인 프롬프트 사용**
   - 요청하는 내용에 대한 컨텍스트 포함
   - 각 선택의 결과를 설명
   - 질문을 간단하고 명확하게 유지

3. **예상치 못한 입력 처리**
   - 사용자 응답 유효성 검사
   - 잘못된 입력에 대한 기본값 제공
   - 명확한 오류 메시지 제공

4. **요청 ID 추적**
   - request_id와 응답 간의 상관관계 사용
   - 상태를 수동으로 관리하려고 하지 않기

5. **비차단 설계**
   - 입력을 기다리며 스레드를 차단하지 않기
   - 비동기 패턴 전반에 걸쳐 사용
   - 동시 워크플로 인스턴스 지원

### 📚 관련 개념:

- **Agent Middleware** - 에이전트 호출 가로채기 (이전 노트북)
- **워크플로 상태 관리** - 실행 간 워크플로 상태 유지
- **다중 에이전트 협업** - human-in-the-loop와 에이전트 팀 결합
- **이벤트 기반 아키텍처** - 이벤트로 반응형 시스템 구축

---

### 🎓 축하합니다!

Microsoft Agent Framework를 사용한 human-in-the-loop 워크플로를 마스터하셨습니다! 이제 다음을 할 수 있습니다:
- ✅ 워크플로를 일시 중지하여 인간 입력 수집
- ✅ RequestInfoExecutor와 RequestInfoMessage 사용
- ✅ 이벤트를 사용한 스트리밍 실행 처리
- ✅ @handler로 사용자 정의 executor 생성
- ✅ 인간 결정에 따라 워크플로 라우팅
- ✅ 인간과 협력하는 대화형 AI 에이전트 구축

**이것은 신뢰할 수 있고 제어 가능한 AI 시스템을 구축하기 위한 중요한 패턴입니다!** 🚀



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임지지 않습니다.
